# Sports Betting Dataset

Current challenge is to build a recommendation engine to suggest betting combinations to users based on various factors. 

At its core, this type of model aims to:

- Analyze historical betting data and outcomes
- Consider factors like team/player performance statistics, odds movements, and user preferences
- Make personalized recommendations for potential bets
- Help users construct betting combinations

## Problem Understanding:

The datasets captures user interactions, specifically focusing on betting behavior. Core actor or entities involved:

- Users: The people placing bets
- Bets: Individual wagers placed by users
- Selections: Parts of a bet
- Events: The actual sports matches/games
- Markets: Different types of bets available for each event

Key Relationship:

- One bet can have multiple selections
- Each selection is tied to: A specific event or A specific market
- The same event can appear in many different betslips
- They might even bet on different markets within the same event

Lets break down each table and explain the likely purpose of each column:


### Users Table
list of registered users
- **player_id**: Unique identifier for each player/bettor
- brand_id: Identifier for the betting platform/company (useful if multiple brands exist)
- player_reg_date: Registration date of the player
- language: Preferred language of the player

### Events Table
List of sporting events available for betting
- event_id: Unique identifier for each sporting event
- start_time: When the event/match starts
- sport_id: Identifier for the type of sport (e.g., 1=football, 2=basketball)
- league_id: Identifier for the specific league/competition
- home_team: Name/ID of the home team
- away_team: Name/ID of the away team

### Bets Table
Transactional table providing the hisotrical data on bets. There is relationship between Users and Events
- bet_id: Unique identifier for each bet
- brand_id: Links to the brand where bet was placed
- player_id: Links to the user who placed the bet
- bet_date: When the bet was placed
- amount: Stake amount
- amount_currency: Currency of the bet
- bet_type: Type of bet (e.g., single, accumulator/parlay)
- bet_odds: Total odds for the bet
- status: Current status of bet (e.g., pending, won, lost)
- settlement_timestamp: When the bet was settled/resulted
- outcome_id: Identifier for the specific outcome chosen
- specifier: Additional bet specifications/parameters
- event_id: Links to the specific event
- market_id: Type of market (e.g., match winner, over/under)
- outcome_odds: Odds for this specific selection

In [18]:
import pandas as pd
from recsys.config import Settings
from recsys.utils.utils import IDConverter
from recsys.features import users, events, interactions, labels, BettingDataset
import pickle

In [19]:
settings = Settings()
SOURCE_DIR = settings.SOURCE_DATA_DIR
processed_dir = settings.PROCESSED_DATA_DIR

In [23]:
# Prepare data
user_df = pd.read_csv(SOURCE_DIR / 'users.csv')
event_df = pd.read_csv(SOURCE_DIR / 'events.csv')
bet_df = pd.read_csv(SOURCE_DIR / 'bets.csv')

In [25]:
# Check for nulls
user_df.isnull().sum()


player_id             0
brand_id              0
player_reg_date       0
language           2074
dtype: int64

In [27]:
# Check for Duplicates
user_df[user_df.duplicated()]

,player_id,brand_id,player_reg_date,language
14,157fb4f9a2dcd04e278c,38a55baabc72e1f7eaa2,2023-05-01 17:37:18.398972 +0000,tur
100,ffde483f3b3023339410,38a55baabc72e1f7eaa2,2023-02-15 16:57:13.660510 +0000,tur
116,681b545f6e7542c249d2,38a55baabc72e1f7eaa2,2024-03-01 15:42:43.071764 +0000,tur
117,681b545f6e7542c249d2,38a55baabc72e1f7eaa2,2024-03-01 15:42:43.071764 +0000,tur
118,835991dd171078841d71,38a55baabc72e1f7eaa2,2024-06-23 21:14:08.572963 +0000,tur
...,...,...,...,...
149116,4524aa46a6f1b9c7d64a,bfd273e31e34e3c263af,2024-09-09 06:12:28.374779 +0000,por
149117,325090c19b36fed3019c,bfd273e31e34e3c263af,2024-10-12 23:28:57.959649 +0000,por
149118,9de4fc1c24b8025ee1e5,bfd273e31e34e3c263af,2024-04-27 18:51:24.369259 +0000,por
149119,aa09731b613f1a911216,bfd273e31e34e3c263af,2024-10-19 19:59:02.664664 +0000,por


In [29]:
user_df_copy = user_df.copy()

In [30]:
user_df_copy['date_len'] = user_df_copy.player_reg_date.str.len()

In [34]:
user_df_copy.date_len.value_counts()

player_reg_date
2024-09-15 12:18:41.199039 +0000        568
2023-04-25 19:45:59.070732 +0000        561
2023-09-17 05:14:34.429611 +0000        516
2021-10-14 13:50:01.294056 +0000        495
2023-12-20 21:34:02.969715 +0000        426
                                       ... 
2023-03-04 17:20:36.459768 +0000 UTC      1
2024-03-20 06:25:23.049966 +0000          1
2024-06-04 08:01:19.427869 +0000 UTC      1
2024-04-08 19:26:01.756678 +0000 UTC      1
2024-10-08 20:49:49.091479 +0000          1
Name: count, Length: 24379, dtype: int64

In [36]:
user_df_copy['player_reg_date'] = pd.to_datetime(user_df_copy.player_reg_date.str[:19])

In [37]:
user_df = user_df.drop_duplicates().copy(deep=True)
event_df = event_df.drop_duplicates().copy(deep=True)
interactions_df = bet_df.drop_duplicates().copy(deep= True)

In [40]:
user_df.columns

Index(['player_id', 'brand_id', 'player_reg_date', 'language'], dtype='object')

In [41]:
event_df.columns

Index(['event_id', 'start_time', 'sport_id', 'league_id', 'home_team',
       'away_team'],
      dtype='object')

In [42]:
interactions_df.columns

Index(['bet_id', 'brand_id', 'player_id', 'bet_date', 'amount',
       'amount_currency', 'bet_type', 'bet_odds', 'status',
       'settlement_timestamp', 'outcome_id', 'specifier', 'event_id',
       'market_id', 'outcome_odds'],
      dtype='object')

In [43]:
interaction_cols = ['bet_id', 'brand_id', 'player_id', 'bet_date', 'amount', 'amount_currency', 'bet_type', 'bet_odds', 'status','event_id','market_id', 'outcome_odds']

In [46]:
interactions_df = interactions_df[interaction_cols].merge(user_df[['player_id', 'brand_id',]], how = 'left', left_on='player_id', right_on = 'player_id')

In [49]:
interactions_df.bet_date = pd.to_datetime(interactions_df.bet_date.str[:19])

In [54]:
interactions_df = interactions_df.merge(event_df[['event_id', 'start_time']], how='left', left_on='event_id', right_on='event_id')

In [ ]:
interactions_df.sort_values(['start_time','bet_date', 'player_id'], ascending=[False, False, False])

,bet_id,brand_id_x,player_id,bet_date,amount,amount_currency,bet_type,bet_odds,status,event_id,market_id,outcome_odds,brand_id_y,start_time
661450,61b21b4fedd1468ea3f8,38a55baabc72e1f7eaa2,67eabe3c0737b3a22262,2024-11-08 17:02:04,51.0,TRY,accumulator,223.86,lose,a56935b8608a53ffc736,acbfceb83cee544eacc7,1.79,38a55baabc72e1f7eaa2,2025-01-08 19:00:00.000
661451,61b21b4fedd1468ea3f8,38a55baabc72e1f7eaa2,67eabe3c0737b3a22262,2024-11-08 17:02:04,51.0,TRY,accumulator,223.86,lose,a56935b8608a53ffc736,acbfceb83cee544eacc7,1.79,38a55baabc72e1f7eaa2,2025-01-08 19:00:00.000
661452,61b21b4fedd1468ea3f8,38a55baabc72e1f7eaa2,67eabe3c0737b3a22262,2024-11-08 17:02:04,51.0,TRY,accumulator,223.86,lose,a56935b8608a53ffc736,acbfceb83cee544eacc7,1.79,38a55baabc72e1f7eaa2,2025-01-08 19:00:00.000
339580,048d7a38930003fc6803,38a55baabc72e1f7eaa2,1b96cfabd2a5eef859d6,2024-11-15 11:11:26,50.0,TRY,accumulator,37.05,cashed out,741a54ca1ce284c7ea8a,acbfceb83cee544eacc7,19.50,38a55baabc72e1f7eaa2,2024-12-21 00:00:00.000
1048776,5ad58c3e7f2d36646294,bfd273e31e34e3c263af,3012b1de31aa925de2dd,2024-11-19 18:23:01,5.0,EUR,ordinar,4.49,lose,6f71637ad614348b8377,afcafb79dd46e8ff57c1,4.49,bfd273e31e34e3c263af,2024-11-19 16:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996614,c11f651cf41bf67d22c0,38a55baabc72e1f7eaa2,48dbfacc8258f0ff0aa4,2024-10-15 06:09:45,100.0,TRY,ordinar,1.90,lose,1f19c92f1783b4acde7b,afcafb79dd46e8ff57c1,1.90,38a55baabc72e1f7eaa2,NaN
996615,c11f651cf41bf67d22c0,38a55baabc72e1f7eaa2,48dbfacc8258f0ff0aa4,2024-10-15 06:09:45,100.0,TRY,ordinar,1.90,lose,1f19c92f1783b4acde7b,afcafb79dd46e8ff57c1,1.90,38a55baabc72e1f7eaa2,NaN
378308,ef96436d40b3d94f656d,38a55baabc72e1f7eaa2,d2cfe5fd89437e11fc11,2024-10-15 03:47:14,2016.0,TRY,ordinar,1.54,win,fd53841ae21e7f2db764,afcafb79dd46e8ff57c1,1.54,38a55baabc72e1f7eaa2,NaN
378309,ef96436d40b3d94f656d,38a55baabc72e1f7eaa2,d2cfe5fd89437e11fc11,2024-10-15 03:47:14,2016.0,TRY,ordinar,1.54,win,fd53841ae21e7f2db764,afcafb79dd46e8ff57c1,1.54,38a55baabc72e1f7eaa2,NaN


In [70]:
interactions_df.groupby(['player_id', pd.Grouper(key='bet_date', freq='M')]).agg({'bet_id':'count'}).unstack()

/var/folders/4x/48wx0c5156g28f5nn0ljxn4w0000gn/T/ipykernel_62248/1347606883.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  interactions_df.groupby(['player_id', pd.Grouper(key='bet_date', freq='M')]).agg({'bet_id':'count'}).unstack()


bet_id           
bet_date             2024-10-31 2024-11-30
player_id                                 
0003b10f0a86cf2d43b0        6.0        5.0
0019e5a5a8e130da2beb        NaN       13.0
0025126efda93a3e00ff        4.0        NaN
0027a74d7c6635cd6de9        NaN        5.0
002ff92bd3c09373ecd1        3.0       20.0
...                         ...        ...
ffc825f3e731e7ad7d84        2.0       21.0
ffcc0622268fcc452735        7.0        NaN
ffde483f3b3023339410      648.0      498.0
ffe678889a516863bd23        7.0        NaN
ffeb3db0c7df6ce0e942        NaN       54.0

[9443 rows x 2 columns]

In [72]:
interactions_df.pivot_table(columns='bet_type', index='market_id', values= 'bet_id', aggfunc='count')